# 인스타그램 - 제주맛집

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
import unicodedata, re
from tqdm.notebook import tqdm

In [3]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

### 1. 로그인

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'ckiekim'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [7]:
input_pwd.submit()
time.sleep(1)

In [8]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [9]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주맛집으로 검색

In [10]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [11]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글

In [32]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [13]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'#제주맛집🍜제주도민선정맛집 ! 제주순메밀막국수🍜🎄직접 짜서 만드는 들기름으로 건강 up! 고소함 up!🎄30년 장인이 직접만든 쯔유,판메밀 전문점! 🎄100% 제주산 순메밀만 사용하는 제주도민맛집!..#카멜리아힐맛집.밀가루 70% 수입메일 30%을 사용하는 가게와는 차별된 🧡국내산 100% 순메밀🧡을 사용하는고객의 건강까지 생각하는 식당☺..순메밀막국수는 코로나 방역지침을 준수하고 있습니다👏..____________________________________주소 : 제주 서귀포시 안덕면 서광리 1413-3전화 : 064-792-0600 📞 ____________________________________#안덕면맛집#제주도서쪽맛집 #제주국제학교맛집 #오설록근처맛집 #제주도오설록맛집#제주도국수#제주도막국수맛집'

In [15]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집🍜제주도민선정맛집',
 '#카멜리아힐맛집.밀가루',
 '#안덕면맛집',
 '#제주도서쪽맛집',
 '#제주국제학교맛집',
 '#오설록근처맛집',
 '#제주도오설록맛집',
 '#제주도국수',
 '#제주도막국수맛집']

In [16]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-08'

In [17]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [18]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'제주순메밀막국수'

In [19]:
row = [content, date, like, place, tags]
row

['#제주맛집🍜제주도민선정맛집 ! 제주순메밀막국수🍜🎄직접 짜서 만드는 들기름으로 건강 up! 고소함 up!🎄30년 장인이 직접만든 쯔유,판메밀 전문점! 🎄100% 제주산 순메밀만 사용하는 제주도민맛집!..#카멜리아힐맛집.밀가루 70% 수입메일 30%을 사용하는 가게와는 차별된 🧡국내산 100% 순메밀🧡을 사용하는고객의 건강까지 생각하는 식당☺..순메밀막국수는 코로나 방역지침을 준수하고 있습니다👏..____________________________________주소 : 제주 서귀포시 안덕면 서광리 1413-3전화 : 064-792-0600 📞 ____________________________________#안덕면맛집#제주도서쪽맛집 #제주국제학교맛집 #오설록근처맛집 #제주도오설록맛집#제주도국수#제주도막국수맛집',
 '2021-12-08',
 0,
 '제주순메밀막국수',
 ['#제주맛집🍜제주도민선정맛집',
  '#카멜리아힐맛집.밀가루',
  '#안덕면맛집',
  '#제주도서쪽맛집',
  '#제주국제학교맛집',
  '#오설록근처맛집',
  '#제주도오설록맛집',
  '#제주도국수',
  '#제주도막국수맛집']]

### 4. 다음 게시글로 이동

In [35]:
# 첫번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [36]:
# 두번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [37]:
# 5를 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 5. 반복문으로 데이터 가져오기

In [38]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [39]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [40]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

In [41]:
driver.close()

### 6. 데이터프레임으로 만들고 중복 제거

In [43]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,#제주맛집🍜제주도민선정맛집 ! 제주순메밀막국수🍜🎄직접 짜서 만드는 들기름으로 건강 ...,2021-12-08,0,제주순메밀막국수,"[#제주맛집🍜제주도민선정맛집, #카멜리아힐맛집.밀가루, #안덕면맛집, #제주도서쪽맛..."
1,"이국적인 바다뷰를 보며 즐기는애월 흑돼지 '코시롱' 🌴애월 바다뷰 노을뷰 맛집,제주...",2021-12-07,0,,[]
2,"#제주맛집산도롱맨도롱----------🍜갈비국수 ""찐"" 맛집✔💕 큼직한 우대갈비가 ...",2021-12-08,0,산도롱맨도롱,"[#제주맛집산도롱맨도롱----------🍜갈비국수, #제주도맛집.대기시간이, #제주..."
3,광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...,2021-12-06,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
4,#제주맛집산도롱맨도롱----------🍜갈비국수 대표맛집 🍜📌 산도롱의 대표메뉴 홍...,2021-12-08,0,산도롱맨도롱,"[#제주맛집산도롱맨도롱----------🍜갈비국수, #성산일출봉맛집.대기시간이, #..."


In [45]:
df.drop_duplicates(subset='content', keep='first', inplace=True)
df.shape

(333, 5)

In [46]:
df.to_csv('인스타-제주맛집.csv', index=False)